## Imports

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow import keras
from keras.models import Model
import boto3
from time import time
import os
import io
from tqdm import tqdm
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy import sqrt
from numpy import argmax
import seaborn as sns
import math
import sklearn
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Load Model

In [ ]:
model = tf.keras.models.load_model('/home/drevital/cs_video_processor/models/suzuyo')

## General Settings

In [ ]:
cloud_dataset = True
obstacle_dataset = 'suzuyo/eval_pairs/obstacle'
no_obstacle_dataset = 'suzuyo/eval_pairs/no_obstacle'
IMG_HEIGHT, IMG_WIDTH = 200, 200
BATCH_SIZE = 32
rand_images = True
NUM_RAND_IMAGES = 10
batch_size = min(BATCH_SIZE, NUM_RAND_IMAGES*2)
obstacle_pairs = []
no_obstacle_pairs = []

## Utility functions for S3 Image Data Generator

In [ ]:
def get_image(imname):
    client = boto3.client('s3')
    bucket = 'obstacles-classification'
    key = imname
    outfile = io.BytesIO()
    client.download_fileobj(bucket, key, outfile)
    outfile.seek(0)
    im = plt.imread(outfile, format='jpg')
    return im

def preprocess_image(im):
    w = im.shape[1]
    im1 = im[:,:w//2]
    im2 = im[:,w//2:]
    sub = cv2.subtract(im1, im2)
    sub = sub.reshape(sub.shape[0], sub.shape[1], 1)
    arr = keras.preprocessing.image.smart_resize(sub,
                                                 (IMG_HEIGHT, IMG_WIDTH),
                                                 interpolation='bilinear')
    arr /= 255.0
    return arr

## Define S3 Image Generator

In [ ]:
def s3_image_generator(obstacle_image_names,
                       no_obstacle_image_names,
                       num_obstale_images,
                       num_no_obstacle_images,
                       batch_size,
                       eval=False):
    s3 = boto3.resource('s3')
    inputs = []
    outputs = [1]*num_obstacle_images + [0]*num_no_obstacle_images
    num_images = num_obstacle_images + num_no_obstacle_images
    
    for image_name in tqdm(obstacle_image_names):
        im = get_image(image_name)
        if eval:
            obstacle_pairs.append(im)
        im = preprocess_image(im)
        inputs.append(im)
        
    for image_name in tqdm(no_obstacle_image_names):
        im = get_image(image_name)
        if eval:
            no_obstacle_pairs.append(im)
        im = preprocess_image(im)
        inputs.append(im)

    for i in range(0, num_images, batch_size):
        x = np.array(inputs[i:i+batch_size])
        y = np.array(outputs[i:i+batch_size])
        yield(x, y)

## Function to read dataset file names from s3

In [ ]:
def get_dataset_image_names(dataset, num_images=None):
    client = boto3.client('s3')
    bucket = 'obstacles-classification'
    image_names = []
    actual_num_images = 0

    paginator = client.get_paginator('list_objects')
    page_iterator = paginator.paginate(Bucket=bucket, Prefix=dataset)

    for page in page_iterator:
        for image_name in page['Contents']:
            if image_name['Key'].split('.')[-1] == 'jpg':
                image_names.append(image_name['Key'])
                actual_num_images += 1
                
    actual_image_names = np.array(image_names)
    
    if num_images:
        np.random.shuffle(image_names)
        actual_image_names = image_names[:num_images]
        actual_num_images = num_images
        
    return actual_image_names, actual_num_images

## Function to display Confusion Matrix

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('No Obstacles Detected (True Negatives): ', cm[0][0])
  print('No Obstacles Incorrectly Detected (False Positives): ', cm[0][1])
  print('Obstacles Missed (False Negatives): ', cm[1][0])
  print('Obstacles Detected (True Positives): ', cm[1][1])
  print('Total Obstacles: ', np.sum(cm[1]))

## Fetch list of files in Evaluation Dataset to serve the S3 file-generator

In [ ]:
num_images = NUM_RAND_IMAGES if rand_images else None
obstacle_image_names, num_obstacle_images = get_dataset_image_names(obstacle_dataset,
                                                                    num_images)
no_obstacle_image_names, num_no_obstacle_images = get_dataset_image_names(no_obstacle_dataset,
                                                                          num_images)
num_obstacles = len(obstacle_image_names)
num_no_obstacles = len(no_obstacle_image_names)
num_images = num_obstacles + num_no_obstacles

## Evaluate model and print metrics

In [ ]:
metrics = model.evaluate_generator(s3_image_generator(obstacle_image_names,
                                                      no_obstacle_image_names,
                                                      num_obstacle_images,
                                                      num_no_obstacle_images,
                                                      batch_size,
                                                      eval=True),
                                   verbose=1)

for name, value in zip(model.metrics_names, metrics):
  print(name, ': ', value)
print()

## Predict with the model and print prediction charts

In [ ]:
labels = np.array([1]*(num_obstacle_images) + [0]*(num_no_obstacle_images))
num_prediction_batches = math.ceil(num_images/batch_size)

predictions = model.predict_generator(s3_image_generator(obstacle_image_names, 
                                                         no_obstacle_image_names, 
                                                         num_obstacle_images,
                                                         num_no_obstacle_images,
                                                         batch_size),
                                      num_prediction_batches,
                                      verbose=1)

plot_cm(labels, predictions) # Default: threshold = 0.5

## Display False Negatives

In [ ]:
false_negatives = [im for i, im in enumerate(obstacle_pairs) if predictions[i] <= 0.5]

if false_negatives:
    num_images = len(false_negatives)
    _, axarr = plt.subplots(num_images, 1, figsize=(1.5*num_images, 1.5*num_images))

    for i, im in enumerate(false_negatives):
        axarr[i].imshow(im, cmap='gray', vmin=0, vmax=255)

In [ ]:
len(axarr)

In [ ]:
num_obstacles

## Display False Positives

In [ ]:
false_positives = [im for i, im in enumerate(no_obstacle_pairs) if predictions[i+NUM_RAND_IMAGES] > 0.5]

if false_positives:
    num_images = len(false_positives)
    _, axarr = plt.subplots(num_images, 1, figsize=(1.5*num_images, 1.5*num_images))

    for i, im in enumerate(false_positives):
        axarr[i].imshow(im, cmap='gray', vmin=0, vmax=255)

## Find Optimal Threshold

In [ ]:
true_values = [1]*NUM_RAND_IMAGES + [0]*NUM_RAND_IMAGES
fpr, tpr, thresholds = roc_curve(true_values, predictions)
gmeans = sqrt(tpr * (1-fpr))
ix = argmax(gmeans)
optimal_threshold = thresholds[ix]
print(f'Optimal Threshold: {optimal_threshold}')

## Use Optimal Threshold

In [ ]:
plot_cm(labels, predictions, p=optimal_threshold) 